# Predicting Performing Arts Attendance with Machine Learning

## - Data Cleaning

August 8, 2025

---

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

## Load data

In [ ]:
data_fname = 'data/cps_00020.csv'
data = pd.read_csv(data_fname)

## Select variables and PPA data

In [ ]:
# Select variables of interest
data_selected = data[[

    # geographics
    'REGION',     # Region and division (nine dividions)
    'STATEFIP',   # State (FIPS code)
    'METRO',      # Metropolitan and central/principal city status
    
    # income
    'FAMINC',     # Family income of householder (in $500 intervals)
    
    # demographics
    'AGE',
    'SEX',
    'RACE',
    'HISPAN',
    'VETSTAT',
    'YRIMMIG',
    'MARST',      # Marital status
    
    # family size
    'FAMSIZE',    # Number of own family members in hh
    'NCHILD',     # Number of own children in household
    'NCHLT5',     # Number of own children under age 5 in hh
    
    # employment
    'EMPSTAT',    # Employment status
    'CLASSWKR',   # Class of worker
    'UHRSWORKT',  # Hours usually worked per week at all jobs
    
    # education
    'EDUC99',     # Educational attainment, 1990
    'SCHLCOLL',
    'PROFCERT',
    
    # difficulties
    'DIFFHEAR',   # hearing
    'DIFFEYE',    # vision
    'DIFFREM',    # memory
    'DIFFPHYS',   # physical
    'DIFFMOB',    # limited mobility
    'DIFFANY',    # any
    
    # live in-person performing arts attendance
    # high-brow
    'PBALLET',    # ballet
    'PCLASS',     # classical music
    'POPERA',     # opera
    'PNMUS',      # nonmusical stage play
    # popular
    'PJAZZ',      # jazz
    'PDANCE',     # dance, not ballet
    'PMUSICAL',   # musical stage play or operetta
    'PFESTIVAL',  # outdoor festival that featured performing artists
    
    # weight
    'PASUPPWT'
    
]]

data_selected.head()

In [ ]:
# Keep PPA participants only
df = data_selected[data_selected['PASUPPWT'] > 0].copy(deep=True)
df.drop(columns=['PASUPPWT'], inplace=True)
df.shape

## Clean and relabel the data

### Y

In [ ]:
# Define variable groups
highbrow_vars = [
    'PBALLET',    # ballet
    'PCLASS',     # classical music
    'POPERA',     # opera
    'PNMUS',      # nonmusical stage play
]
popular_vars = [
    'PJAZZ',      # jazz
    'PDANCE',     # dance, not ballet
    'PMUSICAL',   # musical stage play or operetta
    'PFESTIVAL'   # outdoor festival that featured performing artists
]
art_vars = highbrow_vars + popular_vars

In [ ]:
# Remove rows with invalid responses
df_clean = df[art_vars].apply(lambda row: row.isin([1, 2]).all(), axis=1)
# print(f'{len(df) - df_clean.sum()} rows removed')
df = df[df_clean].copy()
df.shape

In [ ]:
# Recode: 2 (Yes) → 1, 1 (No) → 0
df[art_vars] = df[art_vars].replace({1: 0, 2: 1})

In [ ]:
# Create ATTEND: 1 if any vars == 1, else 0
df['ATTEND'] = (df[art_vars].sum(axis=1) > 0).astype(int)

In [ ]:
# Count attendance
attendance_summary = {
    'Any Performing Arts': df['ATTEND'].value_counts().sort_index()
}

# Label mapping
attend_labels = {0: '0: Did Not Attend', 1: '1: Attended'}

# Plot pie charts
fig_class, ax = plt.subplots(figsize=(5, 5))
title, counts = next(iter(attendance_summary.items()))
counts.index = counts.index.map(attend_labels)
counts.plot.pie(
    ax=ax,
    autopct='%1.1f%%',
    startangle=90,
    counterclock=False,
    colors=['lightblue', 'gold'],
    wedgeprops={'edgecolor': 'black'}
)

ax.set_title(title)
ax.set_ylabel('')

plt.tight_layout()
plt.close()

In [ ]:
df.drop(columns=art_vars, inplace=True)

### X

In [ ]:
region_labels = {
    11: 'New England',
    12: 'Middle Atlantic',
    21: 'East North Central',
    22: 'West North Central',
    31: 'South Atlantic',
    32: 'East South Central',
    33: 'West South Central',
    41: 'Mountain',
    42: 'Pacific',
}

df['REGION'] = df['REGION'].map(region_labels)

In [ ]:
df['REGION'].value_counts(dropna=False)

In [ ]:
statefip_labels = {
    1: "AL",
    2: "AK",
    4: "AZ",
    5: "AR",
    6: "CA",
    8: "CO",
    9: "CT",
    10: "DE",
    11: "DC",
    12: "FL",
    13: "GA",
    15: "HI",
    16: "ID",
    17: "IL",
    18: "IN",
    19: "IA",
    20: "KS",
    21: "KY",
    22: "LA",
    23: "ME",
    24: "MD",
    25: "MA",
    26: "MI",
    27: "MN",
    28: "MS",
    29: "MO",
    30: "MT",
    31: "NE",
    32: "NV",
    33: "NH",
    34: "NJ",
    35: "NM",
    36: "NY",
    37: "NC",
    38: "ND",
    39: "OH",
    40: "OK",
    41: "OR",
    42: "PA",
    44: "RI",
    45: "SC",
    46: "SD",
    47: "TN",
    48: "TX",
    49: "UT",
    50: "VT",
    51: "VA",
    53: "WA",
    54: "WV",
    55: "WI",
    56: "WY",
}

df['STATEFIP'] = df['STATEFIP'].map(statefip_labels)

In [ ]:
df['STATEFIP'].value_counts(dropna=False)

In [ ]:
metro_labels = {
    1: "Not metropolitan",
    2: "Central city",
    3: "Not central city",
    4: "Central city status not identified",
}

df['METRO'] = df['METRO'].map(metro_labels)

In [ ]:
df['METRO'].value_counts(dropna=False)

In [ ]:
faminc_labels = {
    100: "Under $5,000",
    210: "$5,000 - 7,499",
    300: "$7,500 - 9,999",
    430: "$10,000 - 12,499",
    470: "$12,500 - 14,999",
    500: "$15,000 - 19,999",
    600: "$20,000 - 24,999",
    710: "$25,000 - 29,999",
    720: "$30,000 - 34,999",
    730: "$35,000 - 39,999",
    740: "$40,000 - 49,999",
    820: "$50,000 - 59,999",
    830: "$60,000 - 74,999",
    841: "$75,000 - 99,999",
    842: "$100,000 - 149,999",
    843: "$150,000 and over",
}

faminc_ordered_labels = [
    "Under $5,000",
    "$5,000 - 7,499",
    "$7,500 - 9,999",
    "$10,000 - 12,499",
    "$12,500 - 14,999",
    "$15,000 - 19,999",
    "$20,000 - 24,999",
    "$25,000 - 29,999",
    "$30,000 - 34,999",
    "$35,000 - 39,999",
    "$40,000 - 49,999",
    "$50,000 - 59,999",
    "$60,000 - 74,999",
    "$75,000 - 99,999",
    "$100,000 - 149,999",
    "$150,000 and over"
]

faminc_ranked = {label: rank for rank, label in enumerate(faminc_ordered_labels)}

df['FAMINC'] = df['FAMINC'].map(faminc_labels).map(faminc_ranked)

In [ ]:
df['FAMINC'].value_counts(dropna=False)

In [ ]:
sex_labels = {
    1: 'Male', 
    2: 'Female'
}

df['SEX'] = df['SEX'].map(sex_labels)

In [ ]:
df['SEX'].value_counts(dropna=False)

In [ ]:
df['AGE'].describe()

In [ ]:
race_labels = {
    100: "White",
    200: "Black",
    300: "American Indian/Aleut/Eskimo",
    651: "Asian only",
    652: "Hawaiian/Pacific Islander only",
    801: "White-Black",
    802: "White-American Indian",
    803: "White-Asian",
    804: "White-Hawaiian/Pacific Islander",
    805: "Black-American Indian",
    806: "Black-Asian",
    807: "Black-Hawaiian/Pacific Islander",
    808: "American Indian-Asian",
    809: "Asian-Hawaiian/Pacific Islander",
    810: "White-Black-American Indian",
    811: "White-Black-Asian",
    812: "White-American Indian-Asian",
    813: "White-Asian-Hawaiian/Pacific Islander",
    814: "White-Black-American Indian-Asian",
    815: "American Indian-Hawaiian/Pacific Islander",
    816: "White-Black--Hawaiian/Pacific Islander",
    817: "White-American Indian-Hawaiian/Pacific Islander",
    818: "Black-American Indian-Asian",
    819: "White-American Indian-Asian-Hawaiian/Pacific Islander"
}

df['RACE'] = df['RACE'].map(race_labels)

df['RACE'] = df['RACE'].apply(lambda x: x if x in ['White', 'Black', 'Asian only'] or pd.isna(x) else 'Other')

In [ ]:
df['RACE'].value_counts(dropna=False)

In [ ]:
hispan_labels = {
    0: "Not Hispanic",
    100: "Mexican",
    102: "Mexican American",
    103: "Mexicano/Mexicana",
    104: "Chicano/Chicana",
    108: "Mexican (Mexicano)",
    109: "Mexicano/Chicano",
    200: "Puerto Rican",
    300: "Cuban",
    400: "Dominican",
    500: "Salvadoran",
    600: "Other Hispanic",
    610: "Central/South American",
    611: "Central American, (excluding Salvadoran)",
    612: "South American",
}

df['HISPAN'] = df['HISPAN'].map(hispan_labels)

df['HISPAN'] = df['HISPAN'].apply(lambda x: x if x == "Not Hispanic" or pd.isna(x) else "Hispanic")

In [ ]:
df['HISPAN'].value_counts(dropna=False)

In [ ]:
vetstat_labels = {
    0: "NIU",
    1: "No service",
    2: "Yes"
}

df['VETSTAT'] = df['VETSTAT'].map(vetstat_labels)

In [ ]:
df['VETSTAT'].value_counts(dropna=False)

In [ ]:
df['IMMIG'] = df['YRIMMIG'].apply(lambda x: 1 if pd.notna(x) and x > 0 else 0)

In [ ]:
df['IMMIG'].value_counts(dropna=False)

In [ ]:
marst_labels = {
    1: "Married, spouse present",
    2: "Married, spouse absent",
    3: "Separated",
    4: "Divorced",
    5: "Widowed",
    6: "Never married/single"
}

df['MARST'] = df['MARST'].map(marst_labels)

In [ ]:
df['MARST'].value_counts(dropna=False)

In [ ]:
df['FAMSIZE'].describe()

In [ ]:
df['NCHILD'].describe()

In [ ]:
df['NCHLT5'].describe()

In [ ]:
empstat_labels = {
    1: "Armed Forces",
    10: "At work",
    12: "Has job, not at work last week",
    21: "Unemployed, experienced worker",
    22: "Unemployed, new worker",
    32: "NILF, unable to work",
    34: "NILF, other",
    36: "NILF, retired"
}

df['EMPSTAT'] = df['EMPSTAT'].map(empstat_labels)

In [ ]:
df['EMPSTAT'].value_counts(dropna=False)

In [ ]:
classwkr_labels = {
    0: "NIU",
    13: "Self-employed, not incorporated",
    14: "Self-employed, incorporated",
    22: "Private, for profit",
    23: "Private, nonprofit",
    25: "Federal government employee",
    26: "Armed forces",
    27: "State government employee",
    28: "Local government employee",
    29: "Unpaid family worker"
}

df['CLASSWKR'] = df['CLASSWKR'].map(classwkr_labels)

In [ ]:
df['CLASSWKR'].value_counts(dropna=False)

In [ ]:
df['UHRSWORKT'].describe()

In [ ]:
df.loc[df['UHRSWORKT'] >= 997, 'UHRSWORKT'] = 0

In [ ]:
df['UHRSWORKT'].describe()

In [ ]:
educ99_labels = {
    0: 'NIU',
    1: 'No school',
    4: '1st-4th',
    5: '5th-8th',
    6: '9th',
    7: '10th',
    8: '11th',
    9: '12th, no diploma',
    10: 'High school/GED',
    11: 'Some college',
    12: 'Associate',
    13: 'Associate, occupational',
    14: 'Associate, academic',
    15: 'Bachelors',
    16: 'Masters',
    17: 'Professional',
    18: 'Doctorate'
}
educ99y_labels = {
    'NIU': 0,
    'No school': 0,
    '1st-4th': 2.5,
    '5th-8th': 6.5,
    '9th': 9,
    '10th': 10,
    '11th': 11,
    '12th, no diploma': 12,
    'High school/GED': 12,
    'Some college': 13,
    'Associate': 14,
    'Associate, occupational': 14,
    'Associate, academic': 14,
    'Bachelors': 16,
    'Masters': 18,
    'Professional': 20,
    'Doctorate': 22
}
educ99c_labels = {
    'NIU': 'Less than HS',
    'No school': 'Less than HS',
    '1st-4th': 'Less than HS',
    '5th-8th': 'Less than HS',
    '9th': 'Less than HS',
    '10th': 'Less than HS',
    '11th': 'Less than HS',
    '12th, no diploma': 'Less than HS',
    'High school/GED': 'High school',
    'Some college': 'Some college',
    'Associate': 'Associate',
    'Associate, occupational': 'Associate',
    'Associate, academic': 'Associate',
    'Bachelors': 'Bachelors',
    'Masters': 'Masters',
    'Professional': 'Professional',
    'Doctorate': 'Doctorate'
}

df['EDUC99'] = df['EDUC99'].map(educ99_labels)
df['EDUC99Y'] = df['EDUC99'].map(educ99y_labels)
df['EDUC99'] = df['EDUC99'].map(educ99c_labels)

In [ ]:
df['EDUC99'].value_counts(dropna=False)

In [ ]:
df['EDUC99Y'].describe()

In [ ]:
schlcoll_labels = {
    0: "NIU",
    1: "High school FT",
    2: "High school PT",
    3: "College FT",
    4: "College PT",
    5: "Not attend school"
}

df['SCHLCOLL'] = df['SCHLCOLL'].map(schlcoll_labels)

In [ ]:
df['SCHLCOLL'].value_counts(dropna=False)

In [ ]:
profcert_labels = {
    1: 0,
    2: 1
}

df['PROFCERT'] = df['PROFCERT'].map(profcert_labels)

In [ ]:
df['PROFCERT'].value_counts(dropna=False)

In [ ]:
diff_vars = ['DIFFHEAR', 'DIFFEYE', 'DIFFREM', 'DIFFPHYS', 'DIFFMOB', 'DIFFANY']
diff_labels = {
    0: 'NIU', 
    1: 'No difficulty', 
    2: 'Has difficulty'
}

for var in diff_vars:
    df[var] = df[var].map(diff_labels)

In [ ]:
df.reset_index(drop=True, inplace=True)

## Descriptive Stats

In [ ]:
def compute_ci(x):
    """Compute statistics and confidence interval."""
    x = x.dropna()
    n = len(x)
    mean = x.mean()
    sd = x.std(ddof=1)
    se = sd / np.sqrt(n)
    ci = stats.t.interval(0.95, df=n-1, loc=mean, scale=se) if n > 1 else (np.nan, np.nan)
    return pd.Series({
        'mean': mean * 100,
        'se': se * 100,
        'n': n,
        'ci_lower': ci[0] * 100,
        'ci_upper': ci[1] * 100
    })

In [ ]:
df2 = df.copy(deep=True)

In [ ]:
compute_ci(df2['ATTEND']).to_frame().T

In [ ]:
df2.groupby('FAMINC')['ATTEND'].apply(compute_ci).reset_index().pivot(
    index='FAMINC', columns='level_1', values='ATTEND').reset_index()

In [ ]:
df2.groupby('SEX')['ATTEND'].apply(compute_ci).reset_index().pivot(
    index='SEX', columns='level_1', values='ATTEND').reset_index()

In [ ]:
df2['AGE_GROUP'] = pd.cut(
    df2['AGE'],
    bins=[17, 24, 44, 64, np.inf],  # boundaries: (17,24], (24,44], etc.
    labels=['18–24', '25–44', '45–64', '65+'],
    right=True,  # intervals are right-closed by default
    include_lowest=False
)

df2.groupby('AGE_GROUP')['ATTEND'].apply(compute_ci).reset_index().pivot(
    index='AGE_GROUP', columns='level_1', values='ATTEND').reset_index()

In [ ]:
df2.groupby('RACE')['ATTEND'].apply(compute_ci).reset_index().pivot(
    index='RACE', columns='level_1', values='ATTEND').reset_index()

In [ ]:
df2.groupby('EDUC99')['ATTEND'].apply(compute_ci).reset_index().pivot(
    index='EDUC99', columns='level_1', values='ATTEND').reset_index()